# Actividad.
La Empresa decide "migrar" de sql server a postgreSQL. Efectuar la conexion Databricks Community con PostgreSQL en Azure. Efectuar algunas consultas sobre PostgreSQL usando PySpark y Scala. Utiliza una base de datos cualquiera.  
Si la version de community da muchos problemas utilizar Azure Databricks.

### Creación de PosgreSQL en Azure

- Crear un grupo de recursos. Luego crear un recurso > Bases de datos > **Azure PostgreSQL**  
- Configura los detalles:  
  -. Nombre del sesrvidor SQL: `databricks-postgresq-leo` 
  -. Región: Italy North u otra disponible  
  -. Autenticación: Habilita SQL Authentication y configura:  
    1. Usuario: `adminuser`  
    2. Contraseña: `ContrasenaFuerte123`  
- Marca la casilla de "Habilitar acceso a Azure Services" 
- En el portal busca Bases de Datos y crea una con el nombre: `db_postgresql`  

In [0]:
import requests
 
# Obtener la IP pública del nodo
public_ip = requests.get('https://api.ipify.org').text
print(f"La IP pública del nodo es: {public_ip}")

La IP pública del nodo es: 34.214.70.55


### Configurar Databricks Community Edition  
Ve a la pestaña Computey selecciona Create Compute con estos requisitos:  
  - Cluster Name: PSQL_Cluster.  
  - Databricks Runtime Version: 11.3 LTS (Scala 2.12, Spark 3.3.1)  
  - Crear clúster.  
  - Mientras el clúster se esta creando, descarga el controlador JDBC para SQL Server, en este caso usaremos [este](https://jdbc.postgresql.org/download/).  
  - En Databricks subir el controlador a tu workspace o a tu DBFS.
  - Una vez que el cluster esté activo ve al Cluster y en el boton Libraries cargar el controlador haciendo click en `Install New` y le pasas el path donde has guardado el controlador.  

In [0]:
# Configuración de conexión JDBC
jdbcHostname = "databricks-postgresq-leo.postgres.database.azure.com" # Servidor SQL
jdbcPort = 5432
jdbcDatabase = "db_postgresql" # Nombre exacto de tu base de datos
jdbcUsername = "adminuser" # Cambiar por tu usuario configurado
jdbcPassword = "ContrasenaFuerte123" # Cambiar por la contraseña configurada

jdbcUrl = f"jdbc:postgresql://{jdbcHostname}:{jdbcPort}/{jdbcDatabase}?sslmode=require"

# Propiedades de conexión
connectionProperties = {
  "user": jdbcUsername,
  "password": jdbcPassword,
  "driver": "org.postgresql.Driver"
}

In [0]:
try:
    # Intentar realizar la consulta
    query = "(SELECT table_name FROM information_schema.tables WHERE table_schema = 'public') AS tables"
    df_tables = spark.read.jdbc(url=jdbcUrl, table=query, properties=connectionProperties)
    
    # Mostrar las tablas
    df_tables.show()
    
except Exception as e:
    print("Error de conexión:", e)



+----------+
|table_name|
+----------+
+----------+



### Paso 1: Crear la tabla con un DataFrame y escribir en PostgreSQL

Aunque PySpark no tiene un método directo para ejecutar comandos como `CREATE TABLE`, puedes crear un DataFrame y escribir los datos directamente a una tabla de PostgreSQL. Si la tabla no existe, PySpark la creará automáticamente.


In [0]:
import psycopg2

# Establecer la conexión con PostgreSQL
conn = psycopg2.connect(
    host=jdbcHostname,
    port=jdbcPort,
    dbname=jdbcDatabase,
    user=jdbcUsername,
    password=jdbcPassword
)

# Crear un cursor para ejecutar la consulta SQL
cursor = conn.cursor()

# Consulta SQL para crear la tabla
create_table_query = """
CREATE TABLE IF NOT EXISTS products (
    ProductID SERIAL PRIMARY KEY,
    ProductName VARCHAR(100),
    Category VARCHAR(50),
    Price DECIMAL(10, 2),
    StockQuantity INT
);
"""

# Ejecutar la consulta de creación de tabla
cursor.execute(create_table_query)

# Confirmar los cambios
conn.commit()

# Cerrar el cursor y la conexión
cursor.close()
conn.close()

print("Tabla creada con éxito.")



Tabla creada con éxito.


### Paso 2: Escribir el DataFrame en PostgreSQL

Usando el método `df.write.jdbc()`, puedes escribir los datos del DataFrame en PostgreSQL. Si la tabla no existe, PySpark la crea automáticamente. Si ya existe, puedes optar por reemplazar los datos o agregar nuevos registros.


In [0]:
from pyspark.sql import Row

# Crear datos de ejemplo para los productos
product_data = [
    Row(ProductName="Laptop", Category="Electronics", Price=799.99, StockQuantity=50),
    Row(ProductName="Smartphone", Category="Electronics", Price=499.99, StockQuantity=200),
    Row(ProductName="Table", Category="Furniture", Price=150.50, StockQuantity=30),
    Row(ProductName="Headphones", Category="Electronics", Price=89.99, StockQuantity=150),
    Row(ProductName="Coffee Maker", Category="Home Appliances", Price=120.00, StockQuantity=80)
]

# Convertir los datos en un DataFrame de PySpark
df_products = spark.createDataFrame(product_data)

# Insertar los datos en la tabla "products" de PostgreSQL
df_products.write.jdbc(url=jdbcUrl, table="products", mode="append", properties=connectionProperties)

print("Datos insertados con éxito.")


Datos insertados con éxito.


### Paso 3: Esquema de la tabla

PySpark crea la tabla con el esquema definido por el DataFrame. Las columnas del DataFrame serán las mismas que las de la tabla en PostgreSQL, asegurando que el esquema de datos esté alineado entre ambos sistemas.

In [0]:
# Leer los datos desde la tabla "products" en PostgreSQL
df_products_from_db = spark.read.jdbc(url=jdbcUrl, table="products", properties=connectionProperties)

# Mostrar los primeros registros
df_products_from_db.display()


productid,productname,category,price,stockquantity
1,Headphones,Electronics,89.99,150
2,Laptop,Electronics,799.99,50
3,Coffee Maker,Home Appliances,120.00,80
4,Table,Furniture,150.50,30
5,Smartphone,Electronics,499.99,200


Comprobamos, conectandonos en la base de datos por `ssh` y realizamos una consulta.![](files/postgresql/images/query_shell.png)



### Constultas SCALA
Primero, necesitas definir la configuración de conexión JDBC en Scala de forma similar a como lo hiciste en PySpark. Aquí te muestro cómo hacerlo:

In [0]:
%scala
// Configuración de conexión JDBC
val jdbcHostname = "databricks-postgresq-leo.postgres.database.azure.com" // Servidor SQL
val jdbcPort = 5432
val jdbcDatabase = "db_postgresql" // Nombre exacto de tu base de datos
val jdbcUsername = "adminuser" // Cambiar por tu usuario configurado
val jdbcPassword = "ContrasenaFuerte123" // Cambiar por la contraseña configurada

val jdbcUrl = s"jdbc:postgresql://$jdbcHostname:$jdbcPort/$jdbcDatabase?sslmode=require"

// Propiedades de conexión
val connectionProperties = new java.util.Properties()
connectionProperties.put("user", jdbcUsername)
connectionProperties.put("password", jdbcPassword)
connectionProperties.put("driver", "org.postgresql.Driver")


jdbcHostname: String = databricks-postgresq-leo.postgres.database.azure.com
jdbcPort: Int = 5432
jdbcDatabase: String = db_postgresql
jdbcUsername: String = adminuser
jdbcPassword: String = ContrasenaFuerte123
jdbcUrl: String = jdbc:postgresql://databricks-postgresq-leo.postgres.database.azure.com:5432/db_postgresql?sslmode=require
connectionProperties: java.util.Properties = {user=adminuser, password=ContrasenaFuerte123, driver=org.postgresql.Driver}
res2: Object = null

1. Seleccionar todos los productos

In [0]:
%scala
// Consultar todos los productos
val dfProducts = spark.read.jdbc(jdbcUrl, "products", connectionProperties)

// Mostrar los primeros 5 productos
dfProducts.show(5)

+---------+------------+---------------+------+-------------+
productid| productname| category| price|stockquantity|
+---------+------------+---------------+------+-------------+
 1| Headphones| Electronics| 89.99| 150|
 2| Laptop| Electronics|799.99| 50|
 3|Coffee Maker|Home Appliances|120.00| 80|
 4| Table| Furniture|150.50| 30|
 5| Smartphone| Electronics|499.99| 200|
+---------+------------+---------------+------+-------------+

dfProducts: org.apache.spark.sql.DataFrame = [productid: int, productname: string ... 3 more fields]

2. Filtrar productos por categoría "Electronics"

In [0]:
%scala
// Filtrar productos de la categoría "Electronics"
val dfElectronics = dfProducts.filter("Category = 'Electronics'")

// Mostrar los resultados
dfElectronics.show(5)

+---------+-----------+-----------+------+-------------+
productid|productname| category| price|stockquantity|
+---------+-----------+-----------+------+-------------+
 1| Headphones|Electronics| 89.99| 150|
 2| Laptop|Electronics|799.99| 50|
 5| Smartphone|Electronics|499.99| 200|
+---------+-----------+-----------+------+-------------+

dfElectronics: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [productid: int, productname: string ... 3 more fields]

3. Contar el número de productos por categoría

In [0]:
%scala
// Contar el número de productos por categoría
val dfCategoryCount = dfProducts.groupBy("Category").count()

// Mostrar el resultado
dfCategoryCount.show()

+---------------+-----+
 Category|count|
+---------------+-----+
 Electronics| 3|
Home Appliances| 1|
 Furniture| 1|
+---------------+-----+

dfCategoryCount: org.apache.spark.sql.DataFrame = [Category: string, count: bigint]

4. Calcular el precio promedio de los productos

In [0]:
%scala
// Importar funciones de Spark SQL
import org.apache.spark.sql.functions._

// Calcular el precio promedio de los productos
val avgPrice = dfProducts.agg(avg("Price").alias("AveragePrice"))

// Mostrar el resultado
avgPrice.show()

+------------+
AveragePrice|
+------------+
 332.094000|
+------------+

import org.apache.spark.sql.functions._
avgPrice: org.apache.spark.sql.DataFrame = [AveragePrice: decimal(14,6)]

5. Productos con precio mayor a 100

In [0]:
%scala
// Filtrar productos con precio mayor a 100
val dfExpensiveProducts = dfProducts.filter("Price > 100")

// Mostrar los resultados
dfExpensiveProducts.show(5)


+---------+------------+---------------+------+-------------+
productid| productname| category| price|stockquantity|
+---------+------------+---------------+------+-------------+
 2| Laptop| Electronics|799.99| 50|
 3|Coffee Maker|Home Appliances|120.00| 80|
 4| Table| Furniture|150.50| 30|
 5| Smartphone| Electronics|499.99| 200|
+---------+------------+---------------+------+-------------+

dfExpensiveProducts: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [productid: int, productname: string ... 3 more fields]